In [1]:
# =========================
# CELL 0: Clean start (restart kernel recommended) + GPU + seeds
# =========================
import os, random, gc
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change if needed

import torch
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA:", torch.cuda.is_available(), "| device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

gc.collect()
if device.type == "cuda":
    torch.cuda.empty_cache()


CUDA: True | device: cuda
GPU: NVIDIA RTX A6000


In [2]:
# =========================
# CELL 1: Imports + Load manifest (SHHS1 only)
# =========================
import re, math
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from collections import Counter

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, average_precision_score

ROOT = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/")
MANIFEST_PATH = ROOT / "manifest_sleepstaging_planA.csv"

manifest = pd.read_csv(MANIFEST_PATH)
print("Manifest rows:", len(manifest))
print(manifest.groupby(["cohort","split"]).size())

# Use ALL SHHS1 (ignore split col for CV)
df_shhs1_all = manifest[manifest.cohort == "SHHS1"].copy()
print("SHHS1 rows:", len(df_shhs1_all))


Manifest rows: 9868
cohort  split        
MESA    external_test    1856
SHHS1   test              548
        train            4380
        val               548
SHHS2   external_test    2536
dtype: int64
SHHS1 rows: 5476


In [3]:
# =========================
# CELL 2: Subject ID utilities
# =========================
def infer_subject_id_from_path(npz_path: str):
    stem = Path(npz_path).stem
    stem = re.sub(r"_inhouse$", "", stem)
    return stem

def ensure_subject_id(df):
    df = df.copy()
    if "subject_id" not in df.columns:
        df["subject_id"] = df["npz_path"].astype(str).apply(infer_subject_id_from_path)
    return df

df_shhs1_all = ensure_subject_id(df_shhs1_all)
print("Unique subjects (SHHS1):", df_shhs1_all["subject_id"].nunique())


Unique subjects (SHHS1): 5476


In [4]:
# =========================
# CELL 3: Augmentation + normalization
# =========================
FS = 125
T = 3750          # 30s * 125 Hz
NUM_CLASSES = 5
LABELS = {0:"W", 1:"N1", 2:"N2", 3:"N3", 4:"REM"}

class EEGAugment:
    def __init__(self,
                 p_amp=0.5, p_noise=0.5, p_shift=0.5,
                 p_bandstop=0.3, p_freqdrop=0.3,
                 amp_range=(0.8, 1.2),
                 noise_std=0.01,
                 shift_max=125,
                 bandstop_ranges=((49,51), (59,61)),
                 freqdrop_max_bins=12):
        self.p_amp = p_amp
        self.p_noise = p_noise
        self.p_shift = p_shift
        self.p_bandstop = p_bandstop
        self.p_freqdrop = p_freqdrop
        self.amp_range = amp_range
        self.noise_std = noise_std
        self.shift_max = shift_max
        self.bandstop_ranges = bandstop_ranges
        self.freqdrop_max_bins = freqdrop_max_bins

    def _amp_scale(self, x):
        s = np.random.uniform(self.amp_range[0], self.amp_range[1])
        return x * s

    def _gaussian_noise(self, x):
        std = np.std(x, axis=1, keepdims=True) + 1e-6
        noise = np.random.randn(*x.shape).astype(np.float32) * (self.noise_std * std)
        return x + noise

    def _time_shift(self, x):
        shift = np.random.randint(-self.shift_max, self.shift_max+1)
        return np.roll(x, shift=shift, axis=1)

    def _bandstop_fft(self, x, fs=125.0):
        E, T_ = x.shape
        X = np.fft.rfft(x, axis=1)
        freqs = np.fft.rfftfreq(T_, d=1.0/fs)
        for (f1, f2) in self.bandstop_ranges:
            mask = (freqs >= f1) & (freqs <= f2)
            X[:, mask] = 0.0
        y = np.fft.irfft(X, n=T_, axis=1).astype(np.float32)
        return y

    def _freq_dropout(self, x):
        E, T_ = x.shape
        X = np.fft.rfft(x, axis=1)
        Fbins = X.shape[1]
        drop = np.random.randint(1, self.freqdrop_max_bins+1)
        start = np.random.randint(0, max(1, Fbins - drop))
        X[:, start:start+drop] = 0.0
        y = np.fft.irfft(X, n=T_, axis=1).astype(np.float32)
        return y

    def __call__(self, x, fs=125.0):
        if np.random.rand() < self.p_amp:      x = self._amp_scale(x)
        if np.random.rand() < self.p_noise:    x = self._gaussian_noise(x)
        if np.random.rand() < self.p_shift:    x = self._time_shift(x)
        if np.random.rand() < self.p_bandstop: x = self._bandstop_fft(x, fs=fs)
        if np.random.rand() < self.p_freqdrop: x = self._freq_dropout(x)
        return x

def normalize_epochs_zscore(x, eps=1e-6, clip=10.0):
    mu = np.mean(x, axis=1, keepdims=True)
    sd = np.std(x, axis=1, keepdims=True) + eps
    x = (x - mu) / sd
    if clip is not None:
        x = np.clip(x, -clip, clip)
    return x.astype(np.float32)

augment = EEGAugment()
print("Augment + normalize ready.")


Augment + normalize ready.


In [5]:
# =========================
# CELL 4: Dataset + Collate
# =========================
class SleepSequenceDataset(Dataset):
    """
    One item = one subject recording.
    Returns:
      x: (L,1,T) float32
      y: (L,)   int64
      mask: (L,) bool
    """
    def __init__(self, df, mode="train",
                 max_hours=None, min_hours=2.0,
                 augmentor=None,
                 exclude_unknown=True,
                 do_normalize=True,
                 boundary_oversample_p=0.70,
                 boundary_radius=2):
        self.paths = df["npz_path"].tolist()
        self.mode = mode
        self.max_hours = max_hours
        self.min_hours = min_hours
        self.augmentor = augmentor
        self.exclude_unknown = exclude_unknown
        self.do_normalize = do_normalize
        self.boundary_oversample_p = float(boundary_oversample_p)
        self.boundary_radius = int(boundary_radius)

    def __len__(self):
        return len(self.paths)

    def _pick_block_start_boundary_aware(self, y, L):
        E = len(y)
        if E <= L:
            return 0
        if np.random.rand() > self.boundary_oversample_p:
            return np.random.randint(0, E - L + 1)

        trans = np.where(y[1:] != y[:-1])[0] + 1
        if trans.size == 0:
            return np.random.randint(0, E - L + 1)

        t = int(np.random.choice(trans))
        start = t - (L // 2)
        start = max(0, min(start, E - L))
        return int(start)

    def __getitem__(self, idx):
        p = self.paths[idx]
        d = np.load(p, allow_pickle=True)
        x = d["x"].astype(np.float32)    # (E,T)
        y = d["y"].astype(np.int64)      # (E,)

        if self.exclude_unknown:
            keep = (y >= 0)
            x = x[keep]
            y = y[keep]

        if self.do_normalize:
            x = normalize_epochs_zscore(x, eps=1e-6, clip=10.0)

        E = len(y)

        if self.max_hours is not None:
            max_L = int((self.max_hours * 3600) / 30)
            min_L = int((self.min_hours * 3600) / 30)

            L = min(max_L, E)
            if E > L:
                start = self._pick_block_start_boundary_aware(y, L)
                x = x[start:start+L]
                y = y[start:start+L]
                E = L

        if self.mode == "train" and self.augmentor is not None:
            x = self.augmentor(x, fs=FS)

        x_t = torch.from_numpy(x).unsqueeze(1)  # (E,1,T)
        y_t = torch.from_numpy(y).long()        # (E,)
        m_t = torch.ones((E,), dtype=torch.bool)
        return x_t, y_t, m_t

def collate_pad(batch):
    lengths = [b[0].shape[0] for b in batch]
    Lmax = max(lengths)
    xs, ys, ms = [], [], []
    for x, y, m in batch:
        L = x.shape[0]
        padL = Lmax - L
        if padL > 0:
            x = torch.cat([x, torch.zeros((padL,1,T), dtype=x.dtype)], dim=0)
            y = torch.cat([y, torch.zeros((padL,), dtype=y.dtype)], dim=0)
            m = torch.cat([m, torch.zeros((padL,), dtype=torch.bool)], dim=0)
        xs.append(x); ys.append(y); ms.append(m)
    return torch.stack(xs,0), torch.stack(ys,0), torch.stack(ms,0)

# loader knobs
BATCH_SUBJ = 2
NUM_WORKERS = 2
PIN = True


In [6]:
# =========================
# CELL 5: Model definitions (V4)
# =========================
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.1):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x):
        if (not self.training) or self.drop_prob == 0.0:
            return x
        keep = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rand = keep + torch.rand(shape, device=x.device)
        mask = torch.floor(rand)
        return x / keep * mask

class ResConv1D(nn.Module):
    def __init__(self, c_in, c_out, k, s=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(c_in, c_out, k, stride=s, padding=k//2),
            nn.BatchNorm1d(c_out),
            nn.GELU(),
            nn.Conv1d(c_out, c_out, k, padding=k//2),
            nn.BatchNorm1d(c_out),
        )
        self.skip = nn.Conv1d(c_in, c_out, 1, stride=s) if (c_in != c_out or s != 1) else nn.Identity()
        self.act = nn.GELU()

    def forward(self, x):
        return self.act(self.conv(x) + self.skip(x))

class EpochEncoder(nn.Module):
    """
    Input : (B,L,1,T)
    Output: (B,L,384)
    """
    def __init__(self, d_model=384):
        super().__init__()
        self.branch_short = ResConv1D(1, 128, k=7,  s=4)
        self.branch_mid   = ResConv1D(1, 128, k=15, s=4)
        self.branch_long  = ResConv1D(1, 128, k=31, s=4)

        self.freq_proj = nn.Sequential(
            nn.Linear(1876, 256),
            nn.LayerNorm(256),
            nn.GELU(),
        )

        self.fuse = nn.Sequential(
            nn.Linear(128*3 + 256, d_model),
            nn.LayerNorm(d_model),
            nn.GELU(),
        )

    def forward(self, x):
        B, L, _, T_ = x.shape
        x = x.view(B*L, 1, T_)

        zs = self.branch_short(x).mean(-1)
        zm = self.branch_mid(x).mean(-1)
        zl = self.branch_long(x).mean(-1)

        with torch.cuda.amp.autocast(enabled=False):
            xf32 = x.squeeze(1).float()
            Xf = torch.fft.rfft(xf32, dim=-1)
            mag = torch.abs(Xf)[:, :1876]
            mag = torch.log1p(mag)
            mag = mag / (mag.mean(dim=1, keepdim=True) + 1e-6)

        zf = self.freq_proj(mag)
        z = torch.cat([zs, zm, zl, zf.to(zs.dtype)], dim=-1)
        z = self.fuse(z)
        return z.view(B, L, -1)

def rotate_half(x):
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RoPE(nn.Module):
    def __init__(self, head_dim, base=10000):
        super().__init__()
        assert head_dim % 2 == 0
        self.head_dim = head_dim
        self.base = base

    def forward(self, x):
        B, L, H, Dh = x.shape
        half = Dh // 2
        freqs = 1.0 / (self.base ** (torch.arange(half, device=x.device) / half))
        t = torch.arange(L, device=x.device)
        angles = torch.einsum("l,d->ld", t, freqs)
        cos = torch.cos(angles)[None, :, None, :]
        sin = torch.sin(angles)[None, :, None, :]
        cos = cos.repeat_interleave(2, dim=-1)
        sin = sin.repeat_interleave(2, dim=-1)
        return (x * cos) + (rotate_half(x) * sin)

class MultiHeadSelfAttentionRoPE(nn.Module):
    def __init__(self, d_model=384, n_heads=8, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.qkv = nn.Linear(d_model, 3*d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.rope = RoPE(self.d_head)

    def forward(self, x, key_padding_mask=None):
        B, L, D = x.shape
        qkv = self.qkv(x)
        q, k, v = qkv.chunk(3, dim=-1)

        q = q.view(B, L, self.n_heads, self.d_head)
        k = k.view(B, L, self.n_heads, self.d_head)
        v = v.view(B, L, self.n_heads, self.d_head)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.d_head)
        scores = scores.float()
        if key_padding_mask is not None:
            scores = scores.masked_fill(~key_padding_mask[:, None, None, :], -1e9)

        attn = torch.softmax(scores, dim=-1)
        attn = self.drop(attn).to(v.dtype)
        out = attn @ v
        out = out.transpose(1, 2).contiguous().view(B, L, D)
        return self.proj(out)

class TransformerBlock(nn.Module):
    def __init__(self, d_model=384, n_heads=8, drop=0.1, drop_path=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadSelfAttentionRoPE(d_model, n_heads, drop)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(4*d_model, d_model),
        )
        self.dp = DropPath(drop_path)

    def forward(self, x, mask):
        x = x + self.dp(self.attn(self.ln1(x), mask))
        x = x + self.dp(self.mlp(self.ln2(x)))
        return x

class HierSleepTransformerV4(nn.Module):
    def __init__(self, num_classes=5, d_model=384, depth=10, n_heads=8):
        super().__init__()
        self.num_classes = num_classes
        self.encoder = EpochEncoder(d_model)
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads, drop_path=0.1*(i+1)/depth)
            for i in range(depth)
        ])
        self.head = nn.Linear(d_model, num_classes)
        self.aux_n1 = nn.Linear(d_model, 2)
        self.trans_logits = nn.Parameter(torch.zeros(num_classes, num_classes))

    def forward(self, x, mask):
        z = self.encoder(x)
        for blk in self.blocks:
            z = blk(z, mask)
        main_logits = self.head(z)     # (B,L,C)
        aux_logits  = self.aux_n1(z)   # (B,L,2)
        return main_logits, aux_logits

tmp = HierSleepTransformerV4(NUM_CLASSES, 384, 10, 8).to(device)
print("Params (M):", sum(p.numel() for p in tmp.parameters())/1e6)
del tmp


Params (M): 19.353504


In [7]:
# =========================
# CELL 6: Loss + EMA + eval (fold-safe, no globals)
# =========================
USE_EMA = True
EMA_DECAY = 0.999

USE_AUX_N1 = True
AUX_N1_WEIGHT = 0.30

USE_LA_CE = True
LA_TAU = 1.0

USE_HARD_NEG_N1 = True
HARD_NEG_MULT = 2.0

USE_TRANS_LOSS = True
TRANS_LOSS_WEIGHT = 0.10

USE_LEARNED_SMOOTHING = True

smooth_vec = torch.tensor([0.02, 0.00, 0.05, 0.05, 0.02], dtype=torch.float32).to(device)

class EMA:
    def __init__(self, model, decay=0.999):
        self.decay = float(decay)
        self.shadow = {n: p.data.detach().clone() for n,p in model.named_parameters() if p.requires_grad}

    @torch.no_grad()
    def update(self, model):
        for name, p in model.named_parameters():
            if name in self.shadow:
                self.shadow[name].mul_(self.decay)
                self.shadow[name].add_((1.0 - self.decay) * p.data)

    def apply(self, model):
        self.backup = {}
        for name, p in model.named_parameters():
            if name in self.shadow:
                self.backup[name] = p.data.detach().clone()
                p.data.copy_(self.shadow[name])

    def restore(self, model):
        for name, p in model.named_parameters():
            if name in self.backup:
                p.data.copy_(self.backup[name])

class LogitAdjustedCE(nn.Module):
    def __init__(self, class_freq, tau=1.0):
        super().__init__()
        freq = torch.tensor(class_freq, dtype=torch.float32)
        self.register_buffer("log_prior", torch.log(freq / freq.sum()))
        self.tau = float(tau)

    def forward(self, logits, targets, reduction="none"):
        logits = logits + self.tau * self.log_prior
        return F.cross_entropy(logits, targets, reduction=reduction)

def label_smoothing_nll(logits, targets, smooth_per_class):
    logp = F.log_softmax(logits, dim=-1)
    nll = -logp.gather(dim=-1, index=targets.view(-1,1)).squeeze(1)
    smooth = -logp.mean(dim=-1)
    s = smooth_per_class[targets]
    return (1 - s) * nll + s * smooth

def compute_transition_loss(model, y, mask):
    B, L = y.shape
    y_prev = y[:, :-1]
    y_next = y[:, 1:]
    m_pair = mask[:, :-1] & mask[:, 1:]
    if m_pair.sum().item() == 0:
        return torch.zeros((), device=y.device)

    y_prev_v = y_prev[m_pair]
    y_next_v = y_next[m_pair]
    logits_pair = model.trans_logits[y_prev_v]  # (N,C)
    return F.cross_entropy(logits_pair, y_next_v)

def apply_learned_smoothing_probs(probs, model):
    Tm = torch.softmax(model.trans_logits, dim=1)  # rows sum to 1
    return probs @ Tm

# These will be set per fold
la_ce = None

def masked_loss_v4(model, main_logits, aux_logits, y, mask):
    B, L, C = main_logits.shape
    logits2 = main_logits.view(B*L, C)
    y2 = y.view(B*L)
    m2 = mask.view(B*L)

    logits_valid = logits2[m2]
    y_valid = y2[m2]

    if USE_LA_CE:
        adj_logits = logits_valid + la_ce.tau * la_ce.log_prior
        loss_main_vec = label_smoothing_nll(adj_logits, y_valid, smooth_vec)
    else:
        loss_main_vec = label_smoothing_nll(logits_valid, y_valid, smooth_vec)

    if USE_HARD_NEG_N1:
        with torch.no_grad():
            pred = torch.argmax(logits_valid, dim=-1)
            hard = (y_valid == 1) & ((pred == 0) | (pred == 2))
        loss_main_vec = loss_main_vec * torch.where(
            hard,
            torch.tensor(HARD_NEG_MULT, device=device),
            torch.tensor(1.0, device=device)
        )

    loss = loss_main_vec.mean()

    if USE_AUX_N1:
        aux2 = aux_logits.view(B*L, 2)[m2]
        y_aux = (y_valid == 1).long()
        loss_aux = F.cross_entropy(aux2, y_aux)
        loss = loss + AUX_N1_WEIGHT * loss_aux

    if USE_TRANS_LOSS:
        loss_trans = compute_transition_loss(model, y, mask)
        loss = loss + TRANS_LOSS_WEIGHT * loss_trans

    return loss

def _ece_from_probs(y_true, probs, n_bins=15):
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    acc = (pred == y_true).astype(np.float32)
    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi)
        if m.sum() == 0:
            continue
        ece += (m.mean()) * abs(acc[m].mean() - conf[m].mean())
    return float(ece)

def _auroc_auprc_multiclass(y_true, probs, num_classes=5):
    Y = label_binarize(y_true, classes=list(range(num_classes)))
    aurocs, auprcs = [], []
    for c in range(num_classes):
        if Y[:, c].sum() == 0:
            continue
        try:
            aurocs.append(roc_auc_score(Y[:, c], probs[:, c]))
            auprcs.append(average_precision_score(Y[:, c], probs[:, c]))
        except Exception:
            pass
    if len(aurocs) == 0:
        return float("nan"), float("nan")
    return float(np.mean(aurocs)), float(np.mean(auprcs))

@torch.no_grad()
def eval_sequence(model, loader, desc="Eval"):
    model.eval()
    all_true, all_pred, all_probs = [], [], []
    total_loss, total_n = 0.0, 0

    for xb, yb, mb in tqdm(loader, desc=desc, leave=False):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        mb = mb.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            main_logits, aux_logits = model(xb, mb)
            loss = masked_loss_v4(model, main_logits, aux_logits, yb, mb)

        probs = torch.softmax(main_logits.float(), dim=-1)
        if USE_LEARNED_SMOOTHING:
            probs = apply_learned_smoothing_probs(probs, model)

        pred = torch.argmax(probs, dim=-1)

        yv = yb[mb].detach().cpu().numpy()
        pv = pred[mb].detach().cpu().numpy()
        pr = probs[mb].detach().cpu().numpy()

        if yv.size == 0:
            continue

        all_true.append(yv); all_pred.append(pv); all_probs.append(pr)

        n = int(mb.sum().item())
        total_loss += float(loss.item()) * n
        total_n += n

    y_true = np.concatenate(all_true)
    y_pred = np.concatenate(all_pred)
    probs_all = np.concatenate(all_probs)

    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    kappa = cohen_kappa_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(NUM_CLASSES)))

    auroc, auprc = _auroc_auprc_multiclass(y_true, probs_all, num_classes=NUM_CLASSES)
    mean_conf = float(probs_all.max(axis=1).mean())
    ece = _ece_from_probs(y_true, probs_all, n_bins=15)

    f1_per = {LABELS[i]: float(f1_score((y_true==i).astype(int), (y_pred==i).astype(int))) for i in range(NUM_CLASSES)}

    return {
        "loss": total_loss / max(total_n, 1),
        "acc": float(acc),
        "macro_f1": float(mf1),
        "kappa": float(kappa),
        "AUROC": float(auroc),
        "AUPRC": float(auprc),
        "meanConf": mean_conf,
        "ECE": ece,
        "f1_per_class": f1_per,
        "cm": cm,
    }


In [8]:
# =========================
# CELL 7: SHHS1-only 10-fold subject-wise CV (fresh per fold)
# =========================
def class_counts_train(df):
    c = Counter()
    for p in tqdm(df["npz_path"].tolist(), desc="Counting labels", leave=False):
        d = np.load(p, allow_pickle=True)
        y = d["y"].astype(np.int64)
        y = y[y >= 0]
        c.update(y.tolist())
    return np.array([c.get(i, 0) for i in range(NUM_CLASSES)], dtype=np.float64)

def make_loader_from_df(df, mode, batch_size, shuffle, augmentor):
    ds = SleepSequenceDataset(
        df, mode=mode,
        max_hours=4.0 if mode=="train" else None,
        min_hours=2.0,
        augmentor=augmentor if mode=="train" else None,
        exclude_unknown=True,
        do_normalize=True,
        boundary_oversample_p=0.70,
        boundary_radius=2
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS if mode=="train" else 1,
        pin_memory=PIN,
        collate_fn=collate_pad,
        persistent_workers=False
    )

def summarize_metric(fold_results, metric):
    vals = np.array([fr["test"][metric] for fr in fold_results], dtype=np.float64)
    return float(np.nanmean(vals)), float(np.nanstd(vals))

FOLDS = 10
EPOCHS_FOLD = 60

fold_results = []

subjects = np.array(sorted(df_shhs1_all["subject_id"].unique()))
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, te_idx) in enumerate(kf.split(subjects), start=1):
    print("\n" + "="*90)
    tr_subj = set(subjects[tr_idx])
    te_subj = set(subjects[te_idx])

    df_tr_pool = df_shhs1_all[df_shhs1_all.subject_id.isin(tr_subj)].reset_index(drop=True)
    df_test    = df_shhs1_all[df_shhs1_all.subject_id.isin(te_subj)].reset_index(drop=True)

    # inner val (10% of trainpool subjects)
    tr_subjects = np.array(sorted(df_tr_pool["subject_id"].unique()))
    rng = np.random.RandomState(SEED + fold)
    rng.shuffle(tr_subjects)
    n_val = max(1, int(0.10 * len(tr_subjects)))

    val_subj = set(tr_subjects[:n_val])
    train_subj = set(tr_subjects[n_val:])

    df_train = df_tr_pool[df_tr_pool.subject_id.isin(train_subj)].reset_index(drop=True)
    df_val   = df_tr_pool[df_tr_pool.subject_id.isin(val_subj)].reset_index(drop=True)

    # sanity: no overlap
    assert set(df_train.subject_id).isdisjoint(set(df_val.subject_id))
    assert set(df_train.subject_id).isdisjoint(set(df_test.subject_id))
    assert set(df_val.subject_id).isdisjoint(set(df_test.subject_id))

    print(f"Fold {fold}: train_subj={df_train.subject_id.nunique()} val_subj={df_val.subject_id.nunique()} test_subj={df_test.subject_id.nunique()}")

    # loaders
    train_loader = make_loader_from_df(df_train, "train", BATCH_SUBJ, True, augment)
    val_loader   = make_loader_from_df(df_val,   "eval",  1, False, None)
    test_loader  = make_loader_from_df(df_test,  "eval",  1, False, None)

    # ---- fresh objects per fold (NEW start)
    gc.collect()
    if device.type == "cuda":
        torch.cuda.empty_cache()

    model_f = HierSleepTransformerV4(NUM_CLASSES, 384, 10, 8).to(device)

    # fold-specific LA prior
    counts_fold = class_counts_train(df_train)
    global la_ce
    la_ce = LogitAdjustedCE(class_freq=counts_fold, tau=LA_TAU).to(device)

    optimizer = torch.optim.AdamW(model_f.parameters(), lr=2e-4, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=5e-4,
        epochs=EPOCHS_FOLD,
        steps_per_epoch=len(train_loader),
        pct_start=0.15,
        div_factor=20.0,
        final_div_factor=100.0
    )
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))
    ema_f = EMA(model_f, EMA_DECAY) if USE_EMA else None

    best_val = -1.0
    best_state = None

    for epoch in range(1, EPOCHS_FOLD+1):
        model_f.train()

        # droppath warmup
        warm = min(1.0, epoch / 10.0)
        for blk in model_f.blocks:
            blk.dp.drop_prob = 0.10 * warm

        running_loss, n_seen = 0.0, 0
        steps_done = 0

        for xb, yb, mb in tqdm(train_loader, desc=f"Fold{fold} Train ep{epoch}", leave=False):
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            mb = mb.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
                main_logits, aux_logits = model_f(xb, mb)
                loss = masked_loss_v4(model_f, main_logits, aux_logits, yb, mb)

            if not torch.isfinite(loss):
                continue

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model_f.parameters(), 1.0)

            scaler.step(optimizer)      # ? correct order
            scaler.update()
            scheduler.step()            # ? correct order

            if ema_f is not None:
                ema_f.update(model_f)

            n = int(mb.sum().item())
            running_loss += float(loss.item()) * max(n,1)
            n_seen += max(n,1)
            steps_done += 1

        train_loss = running_loss / max(n_seen, 1)

        # VAL with EMA
        if ema_f is not None:
            ema_f.apply(model_f)
        val_m = eval_sequence(model_f, val_loader, desc=f"Fold{fold} VAL")
        if ema_f is not None:
            ema_f.restore(model_f)

        print(f"[Fold {fold}][Ep {epoch:02d}] train_loss={train_loss:.4f} | VAL macroF1={val_m['macro_f1']:.4f} acc={val_m['acc']:.4f}")

        if float(val_m["macro_f1"]) > best_val:
            best_val = float(val_m["macro_f1"])
            best_state = {k: v.detach().cpu().clone() for k, v in model_f.state_dict().items()}

    # TEST best
    model_f.load_state_dict({k: v.to(device) for k, v in best_state.items()})

    if ema_f is not None:
        # optional: evaluate as-is (best_state) OR with EMA shadow (pick one)
        pass

    test_m = eval_sequence(model_f, test_loader, desc=f"Fold{fold} TEST")
    fold_results.append({"fold": fold, "best_val_macroF1": best_val, "test": test_m})

    print(f"[Fold {fold}] BEST VAL macroF1={best_val:.4f} | TEST macroF1={test_m['macro_f1']:.4f} acc={test_m['acc']:.4f} kappa={test_m['kappa']:.4f} AUROC={test_m['AUROC']:.4f}")



Fold 1: train_subj=4436 val_subj=492 test_subj=548


Fold1 Train ep1:   0%|                                 | 0/2218 [00:00<?, ?it/s]/usr/local/anaconda3/envs/akbar1/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[Fold 1][Ep 01] train_loss=0.9118 | VAL macroF1=0.6431 acc=0.7839


[Fold 1][Ep 02] train_loss=0.7881 | VAL macroF1=0.7529 acc=0.8339


[Fold 1][Ep 03] train_loss=0.7587 | VAL macroF1=0.7651 acc=0.8388


...

[Fold 10][Ep 57] train_loss=0.5030 | VAL macroF1=0.8004 acc=0.8577


[Fold 10][Ep 58] train_loss=0.5012 | VAL macroF1=0.8008 acc=0.8583


[Fold 10][Ep 59] train_loss=0.5028 | VAL macroF1=0.8005 acc=0.8578


[Fold 10][Ep 60] train_loss=0.5010 | VAL macroF1=0.8010 acc=0.8584


[Fold 10] BEST VAL macroF1=0.8023 | TEST macroF1=0.8063 acc=0.8637 kappa=0.8108 AUROC=0.9715


In [9]:
# =========================
# CELL 8: Summarize mean ± std across folds (SHHS1 TEST)
# =========================
print("\n" + "="*90)
print("SHHS1 10-fold subject-wise CV (TEST on held-out fold)")

for metric in ["acc", "macro_f1", "kappa", "AUROC", "AUPRC", "ECE", "meanConf"]:
    m, s = summarize_metric(fold_results, metric)
    print(f"{metric:8s}: mean={m:.4f}  std={s:.4f}")



SHHS1 10-fold subject-wise CV (TEST on held-out fold)
acc     : mean=0.8602  std=0.0025
macro_f1: mean=0.8025  std=0.0042
kappa   : mean=0.8056  std=0.0040
AUROC   : mean=0.9698  std=0.0014
AUPRC   : mean=0.8530  std=0.0048
ECE     : mean=0.1714  std=0.0038
meanConf: mean=0.6888  std=0.0054


In [11]:
# %% [markdown]
# CELL: Paper-ready table for all 10 folds (from your fold_results structure)

# %%
import pandas as pd
import numpy as np

# --- choose metrics you want in the main table ---
MAIN_METRICS = ["acc", "macro_f1", "kappa", "AUROC", "AUPRC", "ECE", "meanConf"]

def build_fold_tables_from_fold_results(fold_results, main_metrics=MAIN_METRICS):
    """
    fold_results: list of dicts like:
      {"fold": int, "best_val_macroF1": float, "test": {...metrics..., "f1_per_class": {...}}}
    Returns:
      df_main: fold-wise TEST metrics table + Mean±Std row
      df_f1pc: fold-wise per-class F1 table + Mean±Std row (if available)
    """
    # ---- main fold table ----
    rows_main = []
    rows_f1pc = []

    for fr in fold_results:
        fold_id = fr.get("fold", None)
        best_val = fr.get("best_val_macroF1", np.nan)
        test_m = fr.get("test", {})

        row = {"Fold": int(fold_id) if fold_id is not None else np.nan,
               "best_val_macroF1": float(best_val) if best_val is not None else np.nan}

        for m in main_metrics:
            row[m] = float(test_m.get(m, np.nan))

        rows_main.append(row)

        # per-class F1
        f1pc = test_m.get("f1_per_class", None)
        if isinstance(f1pc, dict):
            row_pc = {"Fold": int(fold_id) if fold_id is not None else np.nan}
            # Keep stable order
            for cls in ["W", "N1", "N2", "N3", "REM"]:
                row_pc[f"F1_{cls}"] = float(f1pc.get(cls, np.nan))
            rows_f1pc.append(row_pc)

    df_main = pd.DataFrame(rows_main).sort_values("Fold").reset_index(drop=True)

    df_f1pc = pd.DataFrame(rows_f1pc).sort_values("Fold").reset_index(drop=True) if rows_f1pc else None

    # ---- add Mean ± Std rows ----
    def add_mean_std_row(df, label="Mean±Std"):
        df2 = df.copy()
        numeric_cols = [c for c in df2.columns if c != "Fold"]
        mean_vals = df2[numeric_cols].astype(float).mean(axis=0)
        std_vals  = df2[numeric_cols].astype(float).std(axis=0, ddof=1)

        row = {"Fold": label}
        for c in numeric_cols:
            row[c] = f"{mean_vals[c]:.4f} ± {std_vals[c]:.4f}"

        return pd.concat([df2, pd.DataFrame([row])], ignore_index=True)

    df_main_out = add_mean_std_row(df_main)

    df_f1pc_out = add_mean_std_row(df_f1pc) if df_f1pc is not None and len(df_f1pc) > 0 else None

    return df_main_out, df_f1pc_out


df_main_table, df_f1pc_table = build_fold_tables_from_fold_results(fold_results)

print("Main (TEST) table:")
display(df_main_table)

if df_f1pc_table is not None:
    print("\nPer-class F1 table:")
    display(df_f1pc_table)

# %%
# --- Export: CSV / Excel / LaTeX (paper-ready) ---
# NOTE: escape=False keeps the "±" symbol in LaTeX

df_main_table.to_csv("SHHS1_10fold_TEST_metrics.csv", index=False)
df_main_table.to_excel("SHHS1_10fold_TEST_metrics.xlsx", index=False)

latex_main = df_main_table.to_latex(index=False, escape=False)
with open("SHHS1_10fold_TEST_metrics.tex", "w") as f:
    f.write(latex_main)

if df_f1pc_table is not None:
    df_f1pc_table.to_csv("SHHS1_10fold_TEST_F1_per_class.csv", index=False)
    df_f1pc_table.to_excel("SHHS1_10fold_TEST_F1_per_class.xlsx", index=False)

    latex_pc = df_f1pc_table.to_latex(index=False, escape=False)
    with open("SHHS1_10fold_TEST_F1_per_class.tex", "w") as f:
        f.write(latex_pc)

print("Saved files:")
print(" - SHHS1_10fold_TEST_metrics.csv / .xlsx / .tex")
if df_f1pc_table is not None:
    print(" - SHHS1_10fold_TEST_F1_per_class.csv / .xlsx / .tex")


Main (TEST) table:


,Fold,best_val_macroF1,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf
0,1,0.807106,0.86372,0.806941,0.810847,0.970358,0.85407,0.167371,0.696349
1,2,0.811386,0.861291,0.804916,0.807825,0.969248,0.847966,0.16748,0.693812
2,3,0.806738,0.858548,0.803561,0.804202,0.969138,0.856141,0.175604,0.682944
3,4,0.809759,0.859747,0.801308,0.8049,0.968792,0.850196,0.171538,0.688215
4,5,0.808249,0.857091,0.794228,0.79925,0.966684,0.842234,0.172354,0.684746
5,6,0.808958,0.858035,0.797977,0.802291,0.970279,0.852194,0.179382,0.678653
6,7,0.793013,0.863233,0.807821,0.810079,0.971572,0.858291,0.167453,0.695782
7,8,0.804291,0.860218,0.803485,0.805342,0.970925,0.858103,0.17149,0.68873
8,9,0.813738,0.856891,0.79827,0.8004,0.969964,0.854177,0.168484,0.68841
9,10,0.802317,0.863706,0.806329,0.810806,0.971504,0.856205,0.173194,0.690517



Per-class F1 table:


,Fold,F1_W,F1_N1,F1_N2,F1_N3,F1_REM
0,1,0.915727,0.523936,0.87473,0.83409,0.886223
1,2,0.918521,0.518718,0.87202,0.828395,0.886924
2,3,0.911616,0.527375,0.868429,0.821331,0.889054
3,4,0.908443,0.509475,0.871918,0.829402,0.887303
4,5,0.908989,0.502735,0.874534,0.800267,0.884615
5,6,0.909163,0.505411,0.871729,0.826074,0.877509
6,7,0.915844,0.533453,0.874733,0.824377,0.890695
7,8,0.909207,0.517553,0.877861,0.825465,0.887339
8,9,0.903806,0.509075,0.872542,0.819133,0.886795
9,10,0.913929,0.518593,0.875365,0.829953,0.893805


Saved files:
 - SHHS1_10fold_TEST_metrics.csv / .xlsx / .tex
 - SHHS1_10fold_TEST_F1_per_class.csv / .xlsx / .tex


In [12]:
# %% [markdown]
# CELL: Paper-style table (Mean±Std ONLY for TEST metrics; keep best_val_macroF1 as mean only)

# %%
import pandas as pd
import numpy as np

MAIN_METRICS = ["acc", "macro_f1", "kappa", "AUROC", "AUPRC", "ECE", "meanConf"]

def build_paper_style_tables(fold_results, test_metrics=MAIN_METRICS):
    rows_main = []
    rows_f1pc = []

    for fr in fold_results:
        fold_id = fr.get("fold", None)
        best_val = fr.get("best_val_macroF1", np.nan)
        test_m = fr.get("test", {})

        row = {"Fold": int(fold_id) if fold_id is not None else np.nan,
               "best_val_macroF1": float(best_val) if best_val is not None else np.nan}
        for m in test_metrics:
            row[m] = float(test_m.get(m, np.nan))
        rows_main.append(row)

        f1pc = test_m.get("f1_per_class", None)
        if isinstance(f1pc, dict):
            row_pc = {"Fold": int(fold_id) if fold_id is not None else np.nan}
            for cls in ["W", "N1", "N2", "N3", "REM"]:
                row_pc[f"F1_{cls}"] = float(f1pc.get(cls, np.nan))
            rows_f1pc.append(row_pc)

    df_main = pd.DataFrame(rows_main).sort_values("Fold").reset_index(drop=True)
    df_f1pc = pd.DataFrame(rows_f1pc).sort_values("Fold").reset_index(drop=True) if rows_f1pc else None

    # --- add paper-style summary row ---
    # best_val_macroF1: mean only (no ±)
    # TEST metrics: mean ± std
    def add_paper_summary_row(df, metric_cols, mean_only_cols=("best_val_macroF1",), label="Mean±Std"):
        df2 = df.copy()

        means = df2[metric_cols + list(mean_only_cols)].astype(float).mean(axis=0)
        stds  = df2[metric_cols].astype(float).std(axis=0, ddof=1)

        row = {"Fold": label}

        # mean-only cols
        for c in mean_only_cols:
            if c in df2.columns:
                row[c] = f"{means[c]:.4f}"   # mean only, no ±

        # mean±std cols
        for c in metric_cols:
            row[c] = f"{means[c]:.4f} ± {stds[c]:.4f}"

        return pd.concat([df2, pd.DataFrame([row])], ignore_index=True)

    df_main_out = add_paper_summary_row(df_main, metric_cols=test_metrics, mean_only_cols=("best_val_macroF1",))

    # Per-class F1 table: mean ± std for all F1_* columns
    if df_f1pc is not None and len(df_f1pc) > 0:
        f1_cols = [c for c in df_f1pc.columns if c != "Fold"]
        df_f1pc_out = add_paper_summary_row(df_f1pc, metric_cols=f1_cols, mean_only_cols=())
    else:
        df_f1pc_out = None

    return df_main_out, df_f1pc_out


df_main_paper, df_f1pc_paper = build_paper_style_tables(fold_results)

print("Paper-style main table (Mean±Std for TEST metrics only):")
display(df_main_paper)

if df_f1pc_paper is not None:
    print("\nPaper-style per-class F1 table:")
    display(df_f1pc_paper)

# %%
# Export paper-style tables
df_main_paper.to_csv("SHHS1_10fold_TEST_metrics_paperstyle.csv", index=False)
df_main_paper.to_excel("SHHS1_10fold_TEST_metrics_paperstyle.xlsx", index=False)

latex_main = df_main_paper.to_latex(index=False, escape=False)
with open("SHHS1_10fold_TEST_metrics_paperstyle.tex", "w") as f:
    f.write(latex_main)

if df_f1pc_paper is not None:
    df_f1pc_paper.to_csv("SHHS1_10fold_TEST_F1_per_class_paperstyle.csv", index=False)
    df_f1pc_paper.to_excel("SHHS1_10fold_TEST_F1_per_class_paperstyle.xlsx", index=False)

    latex_pc = df_f1pc_paper.to_latex(index=False, escape=False)
    with open("SHHS1_10fold_TEST_F1_per_class_paperstyle.tex", "w") as f:
        f.write(latex_pc)

print("Saved paper-style files:")
print(" - SHHS1_10fold_TEST_metrics_paperstyle.csv / .xlsx / .tex")
if df_f1pc_paper is not None:
    print(" - SHHS1_10fold_TEST_F1_per_class_paperstyle.csv / .xlsx / .tex")


Paper-style main table (Mean±Std for TEST metrics only):


,Fold,best_val_macroF1,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf
0,1,0.807106,0.86372,0.806941,0.810847,0.970358,0.85407,0.167371,0.696349
1,2,0.811386,0.861291,0.804916,0.807825,0.969248,0.847966,0.16748,0.693812
2,3,0.806738,0.858548,0.803561,0.804202,0.969138,0.856141,0.175604,0.682944
3,4,0.809759,0.859747,0.801308,0.8049,0.968792,0.850196,0.171538,0.688215
4,5,0.808249,0.857091,0.794228,0.79925,0.966684,0.842234,0.172354,0.684746
5,6,0.808958,0.858035,0.797977,0.802291,0.970279,0.852194,0.179382,0.678653
6,7,0.793013,0.863233,0.807821,0.810079,0.971572,0.858291,0.167453,0.695782
7,8,0.804291,0.860218,0.803485,0.805342,0.970925,0.858103,0.17149,0.68873
8,9,0.813738,0.856891,0.79827,0.8004,0.969964,0.854177,0.168484,0.68841
9,10,0.802317,0.863706,0.806329,0.810806,0.971504,0.856205,0.173194,0.690517



Paper-style per-class F1 table:


,Fold,F1_W,F1_N1,F1_N2,F1_N3,F1_REM
0,1,0.915727,0.523936,0.87473,0.83409,0.886223
1,2,0.918521,0.518718,0.87202,0.828395,0.886924
2,3,0.911616,0.527375,0.868429,0.821331,0.889054
3,4,0.908443,0.509475,0.871918,0.829402,0.887303
4,5,0.908989,0.502735,0.874534,0.800267,0.884615
5,6,0.909163,0.505411,0.871729,0.826074,0.877509
6,7,0.915844,0.533453,0.874733,0.824377,0.890695
7,8,0.909207,0.517553,0.877861,0.825465,0.887339
8,9,0.903806,0.509075,0.872542,0.819133,0.886795
9,10,0.913929,0.518593,0.875365,0.829953,0.893805


Saved paper-style files:
 - SHHS1_10fold_TEST_metrics_paperstyle.csv / .xlsx / .tex
 - SHHS1_10fold_TEST_F1_per_class_paperstyle.csv / .xlsx / .tex
